In [1288]:
import numpy as np

def sigmoid(x):
    return 1/(1 + np.exp(-x))

def softmax(x):
    return np.exp(x) / np.sum(np.exp(x)) 

def relu(x):
    return x * (x > 0)

In [1289]:
class Neuron:
    def __init__(self, weight, bias):
        self.weight = weight
        self.bias = bias
        
    def feedforward(self, inputs, act_type):
        total = np.dot(self.weight, inputs) + self.bias
        if act_type == 0:
            return sigmoid(total)
        elif act_type == 1:
            return softmax(total)
        elif act_type == 2:
            return relu(total)

In [1290]:
class Neural_Net:
    def __init__(self, h_num, o_num, i_weight, o_weight, i_bias, o_bias):
        self.i_weight = i_weight
        self.o_weight = o_weight
        self.i_bias = i_bias
        self.o_bias = o_bias
        self.h_num = h_num
        self.o_num = o_num
        self.hidden = []
        self.output = []
        for i in range(h_num):
            self.hidden.append(Neuron(i_weight[i,:], i_bias[i]))
        for i in range(o_num):
            self.output.append(Neuron(o_weight[i,:], o_bias[i]))
    def feedforward(self, i_data):
        o_hidden = []
        o_output = []
        for i in range(self.h_num):
            o_hidden.append(self.hidden[i].feedforward(i_data,0))
        for i in range(self.o_num):
            o_output.append(self.output[i].feedforward(o_hidden,0))
        return o_output, o_hidden
    def backpropagation(self, i_y, y_pred, h_value, i_x):
        h_field = np.matmul(np.matmul(deriv_loss(i_y,y_pred), deriv_pred(h_value,self.o_weight,self.o_bias)), deriv_hidden(i_x, self.i_weight, self.i_bias))
        o_field = np.matmul(np.transpose(deriv_loss(i_y,y_pred)), deriv_hidden(h_value, self.o_weight, self.o_bias))
        h_bias_field = h_field[len(h_field)-self.h_num:]
        h_field = np.reshape(h_field[0:len(h_field)-self.h_num], (self.h_num, len(i_x)))#h_field[0:len(h_field)-self.h_num].reshape[self.h_num, self.o_num]
        o_bias_field = o_field[len(o_field)-self.o_num:]
        o_field = np.reshape(o_field[0:len(o_field)-self.o_num], (self.o_num, self.h_num))
        
        print("dL/dPred",deriv_loss(i_y,y_pred))
        print("dPred/dH",deriv_pred(h_value,self.o_weight,self.o_bias))
        print("dH/dW",deriv_hidden(i_x, self.i_weight, self.i_bias))
        print("dPred/dW",deriv_hidden(h_value, self.o_weight, self.o_bias))
        print("\n")
        
        return h_field, o_field, h_bias_field, o_bias_field

In [1291]:
def deriv_sigmoid(x):
    fx = sigmoid(x)
    return fx * (1 - fx)

def deriv_loss(y_true, y_pred):
    return -2*(y_true - y_pred)

def deriv_pred(hidden, o_weight, o_bias):
    return o_weight * deriv_sigmoid(np.dot(o_weight, hidden) + o_bias)[:,None]

def deriv_hidden(i_data, i_weight, i_bias):
    n_dim = i_weight.shape[0]
    w_dim = i_weight.shape[1]
    derv = np.zeros((n_dim, n_dim * w_dim + n_dim))
    for i in range(0,n_dim):
        derv[i : i+1, i*w_dim : (i+1)*w_dim] = np.array(i_data) * deriv_sigmoid(np.dot(i_weight[i,:],i_data) + i_bias[i])
        derv[i : i+1, n_dim*w_dim+i : n_dim*w_dim+i+1] = deriv_sigmoid(np.dot(i_weight[i,:],i_data) + i_bias[i])
    return derv

In [1292]:
i_weight = np.array([[0.3,0.7],[0.1,0.2]])
o_weight = np.array([[0.4,0.3]])
i_bias = np.array([0.5,0.4])
o_bias = np.array([0.3])
#i_weight = np.array([[0.3,0.7],[0.1,0.2],[0.4,0.1],[0.2,0.5]])
#o_weight = np.array([[0.4,0.3,0.2,0.1]])
#i_bias = np.array([0.5,0.4,0.3,0.5])
#o_bias = np.array([1])
#i_data = np.array([[1,0,1,0],[1,0,1,1]])
i_data = np.array([
  [-2, -1, 1],  # Alice
  [25, 6, 0],   # Bob
  [17, 4, 0],   # Charlie
  [-15, -6, 1], # Diana
])

#x_i_weight = 2
#y_i_weight = 2
#i_weight = np.random.normal(size=(x_i_weight,y_i_weight))

#x_o_weight = 1
#y_o_weight = 2
#o_weight = np.random.normal(size=(x_o_weight,y_o_weight))

#x_i_bias = 1
#y_i_bias = 2
#i_bias = np.random.normal(size=(y_i_bias))

#x_o_bias = 1
#y_o_bias = 1
#o_bias = np.random.normal(size=(x_o_bias))

print(i_weight)
print(o_weight)
print(i_bias)
print(o_bias)

[[0.3 0.7]
 [0.1 0.2]]
[[0.4 0.3]]
[0.5 0.4]
[0.3]


In [1293]:
def train(i_weight, o_weight, i_bias, o_bias, i_data, learning_rate):
    epoch = 1000
    h_size = i_weight.shape[0]
    o_size = i_data.shape[1]
    n = i_data.shape[1]
    obs_n = i_data.shape[0]
    i_y = i_data[: , n-1:n]
    o_size = i_y.shape[1]
    i_x = i_data[: , 0:n-1]
    
    for j in range(epoch):
        loss = 0
        for i in range(0,obs_n):
            network = Neural_Net(h_size, o_size, i_weight, o_weight, i_bias, o_bias)
            pred_y, h_value = network.feedforward(i_x[i, :])
            h_field, o_field, h_bias_field, o_bias_field = network.backpropagation(i_y[i], pred_y, h_value, i_x[i, :])
            i_weight, o_weight, i_bias, o_bias = gradient_descent(i_weight, o_weight, i_bias, o_bias, h_field, o_field, h_bias_field, o_bias_field, learning_rate)
            loss = loss + mse_loss(i_y[i], pred_y)
        print("Epoch %d loss: %.3f" % (epoch, loss))
    #return i_weight, o_weight, i_bias, o_bias
    
        
def gradient_descent(i_weight, o_weight, i_bias, o_bias, h_field, o_field, h_bias_field, o_bias_field, learning_rate):
    i_weight = i_weight - learning_rate * h_field
    o_weight = o_weight - learning_rate * o_field
    i_bias = i_bias - learning_rate * h_bias_field
    o_bias = o_bias - learning_rate * o_bias_field
    return i_weight, o_weight, i_bias, o_bias

def mse_loss(y_true, y_pred):
    return ((y_true - y_pred) ** 2).mean()

In [1294]:
train(i_weight, o_weight, i_bias, o_bias, i_data, 0.1)

dL/dPred [-0.72062401]
dPred/dH [[0.09219491 0.06914618]]
dH/dW [[-0.42781939 -0.2139097   0.          0.          0.2139097   0.        ]
 [ 0.          0.         -0.5        -0.25        0.          0.25      ]]
dPred/dW [[0.07145693 0.11524363 0.23048726]]


dL/dPred [1.47176835]
dPred/dH [[0.07874428 0.05992169]]
dH/dW [[1.35984359e-04 3.26362461e-05 0.00000000e+00 0.00000000e+00
  5.43937436e-06 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 4.28358553e-01 1.02806053e-01
  0.00000000e+00 1.71343421e-02]]
dPred/dW [[0.1943576  0.19096934 0.19435866]]


dL/dPred [1.43349351]
dPred/dH [[0.07644637 0.05688613]]
dH/dW [[4.02734085e-03 9.47609611e-04 0.00000000e+00 0.00000000e+00
  2.36902403e-04 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 9.30169820e-01 2.18863487e-01
  0.00000000e+00 5.47158718e-02]]
dPred/dW [[0.20297274 0.19122729 0.20302085]]


dL/dPred [-0.85729153]
dPred/dH [[0.08509309 0.0619095 ]]
dH/dW [[-4.34296385e-03 -1.73718554e-03  0.00000000e+00  0.00000000e+00
   

dL/dPred [0.73051042]
dPred/dH [[-0.14535797  0.16228172]]
dH/dW [[0.00196227 0.00046171 0.         0.         0.00011543 0.        ]
 [0.         0.         0.08942139 0.02104033 0.         0.00526008]]
dPred/dW [[0.23181708 0.001226   0.23184384]]


dL/dPred [-0.64166246]
dPred/dH [[-0.14030476  0.15250104]]
dH/dW [[-0.002115   -0.000846    0.          0.          0.000141    0.        ]
 [ 0.          0.         -0.07367392 -0.02946957  0.          0.00491159]]
dPred/dW [[3.07280838e-05 2.16823013e-01 2.17898551e-01]]


Epoch 1000 loss: 0.537
dL/dPred [-0.79581634]
dPred/dH [[-0.15426319  0.17100652]]
dH/dW [[-0.40961845 -0.20480923  0.          0.          0.20480923  0.        ]
 [ 0.          0.         -0.38011777 -0.19005889  0.          0.19005889]]
dPred/dW [[0.068859   0.17844403 0.23957726]]


dL/dPred [0.73102499]
dPred/dH [[-0.14805741  0.16882934]]
dH/dW [[4.06164807e-05 9.74795537e-06 0.00000000e+00 0.00000000e+00
  1.62465923e-06 0.00000000e+00]
 [0.00000000e+00 0.0000

dPred/dW [[1.89760256e-01 7.25161860e-05 1.89762273e-01]]


dL/dPred [-0.42542061]
dPred/dH [[-0.1944623   0.20801474]]
dH/dW [[-1.94930107e-04 -7.79720428e-05  0.00000000e+00  0.00000000e+00
   1.29953405e-05  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00 -5.55855133e-03 -2.22342053e-03
   0.00000000e+00  3.70570089e-04]]
dPred/dW [[2.17628820e-06 1.67402552e-01 1.67464632e-01]]


Epoch 1000 loss: 0.269
dL/dPred [-0.60195726]
dPred/dH [[-0.2443084   0.26283306]]
dH/dW [[-0.33767169 -0.16883585  0.          0.          0.16883585  0.        ]
 [ 0.          0.         -0.30139455 -0.15069727  0.          0.15069727]]
dPred/dW [[0.04525642 0.1714942  0.21039049]]


dL/dPred [0.5101624]
dPred/dH [[-0.22013018  0.23933999]]
dH/dW [[1.17501764e-06 2.82004235e-07 0.00000000e+00 0.00000000e+00
  4.70007058e-08 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 1.62972067e-04 3.91132960e-05
  0.00000000e+00 6.51888266e-06]]
dPred/dW [[1.90014772e-01 1.23869213e-06 1.90014781e-01]]


dL/dPred

dL/dPred [-0.31954865]
dPred/dH [[-0.19968648  0.21265181]]
dH/dW [[-2.98080679e-05 -1.19232271e-05  0.00000000e+00  0.00000000e+00
   1.98720452e-06  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00 -9.11458140e-04 -3.64583256e-04
   0.00000000e+00  6.07638760e-05]]
dPred/dW [[2.66775760e-07 1.34238331e-01 1.34246489e-01]]


Epoch 1000 loss: 0.160
dL/dPred [-0.46703003]
dPred/dH [[-0.26623441  0.28428836]]
dH/dW [[-0.27972869 -0.13986434  0.          0.          0.13986434  0.        ]
 [ 0.          0.         -0.2476091  -0.12380455  0.          0.12380455]]
dPred/dW [[0.03009342 0.15307577 0.17898575]]


dL/dPred [0.39644569]
dPred/dH [[-0.23617969  0.2535703 ]]
dH/dW [[7.50744131e-08 1.80178592e-08 0.00000000e+00 0.00000000e+00
  3.00297653e-09 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 1.15263418e-05 2.76632202e-06
  0.00000000e+00 4.61053670e-07]]
dPred/dW [[1.58930547e-01 7.32755460e-08 1.58930548e-01]]


dL/dPred [0.39248545]
dPred/dH [[-0.23539169  0.25165727]]
dH/dW [[



dL/dPred [0.32551738]
dPred/dH [[-0.23433377  0.24879041]]
dH/dW [[7.11878075e-06 1.67500723e-06 0.00000000e+00 0.00000000e+00
  4.18751809e-07 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 2.94900245e-04 6.93882930e-05
  0.00000000e+00 1.73470733e-05]]
dPred/dW [[1.36268242e-01 2.36389718e-06 1.36268299e-01]]


dL/dPred [-0.25952828]
dPred/dH [[-0.19469308  0.20617236]]
dH/dW [[-7.68055218e-06 -3.07222087e-06  0.00000000e+00  0.00000000e+00
   5.12036812e-07  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00 -2.53281876e-04 -1.01312750e-04
   0.00000000e+00  1.68854584e-05]]
dPred/dW [[5.78219964e-08 1.12923503e-01 1.12925410e-01]]


Epoch 1000 loss: 0.107
dL/dPred [-0.38086557]
dPred/dH [[-0.26579909  0.28192266]]
dH/dW [[-0.24018829 -0.12009414  0.          0.          0.12009414  0.        ]
 [ 0.          0.         -0.21257645 -0.10628823  0.          0.10628823]]
dPred/dW [[0.02151809 0.13552819 0.15416814]]


dL/dPred [0.32571317]
dPred/dH [[-0.23494032  0.25001421]]
dH/dW 

...

dL/dPred [0.08329455]
dPred/dH [[-0.12723316  0.13066958]]
dH/dW [[7.85730658e-09 1.84877802e-09 0.00000000e+00 0.00000000e+00
  4.62194505e-10 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 5.00455108e-07 1.17754143e-07
  0.00000000e+00 2.94385358e-08]]
dPred/dW [[3.99127806e-02 1.17497385e-09 3.99127806e-02]]


dL/dPred [-0.0694237]
dPred/dH [[-0.10682388  0.10969762]]
dH/dW [[-8.47739551e-09 -3.39095820e-09  0.00000000e+00  0.00000000e+00
   5.65159701e-10  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00 -4.29907699e-07 -1.71963080e-07
   0.00000000e+00  2.86605133e-08]]
dPred/dW [[1.89367695e-11 3.35069342e-02 3.35069351e-02]]


Epoch 1000 loss: 0.007
dL/dPred [-0.09390577]
dPred/dH [[-0.14266265  0.14651093]]
dH/dW [[-0.09684967 -0.04842483  0.          0.          0.04842483  0.        ]
 [ 0.          0.         -0.08887474 -0.04443737  0.          0.04443737]]
dPred/dW [[0.00228345 0.0426626  0.04474831]]


dL/dPred [0.08329529]
dPred/dH [[-0.12724666  0.13069597]]
dH/dW [[4

dL/dPred [0.08211564]
dPred/dH [[-0.12609646  0.12948955]]
dH/dW [[4.05231404e-13 9.72555370e-14 0.00000000e+00 0.00000000e+00
  1.62092562e-14 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 1.32314986e-10 3.17555967e-11
  0.00000000e+00 5.29259946e-12]]
dPred/dW [[3.93720756e-02 2.08380626e-13 3.93720756e-02]]


dL/dPred [0.08206475]
dPred/dH [[-0.12603438  0.12941273]]
dH/dW [[7.42419081e-09 1.74686843e-09 0.00000000e+00 0.00000000e+00
  4.36717107e-10 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 4.74412711e-07 1.11626520e-07
  0.00000000e+00 2.79066301e-08]]
dPred/dW [[3.93487173e-02 1.09809013e-09 3.93487173e-02]]


dL/dPred [-0.06844436]
dPred/dH [[-0.10587347  0.10870045]]
dH/dW [[-8.01009941e-09 -3.20403976e-09  0.00000000e+00  0.00000000e+00
   5.34006627e-10  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00 -4.07536419e-07 -1.63014567e-07
   0.00000000e+00  2.71690946e-08]]
dPred/dW [[1.76494656e-11 3.30510226e-02 3.30510235e-02]]


Epoch 1000 loss: 0.007
dL/dPred [-0.092

dL/dPred [-0.09113815]
dPred/dH [[-0.13996459  0.14368449]]
dH/dW [[-0.09528734 -0.04764367  0.          0.          0.04764367  0.        ]
 [ 0.          0.         -0.08750971 -0.04375485  0.          0.04375485]]
dPred/dW [[0.00218157 0.04149806 0.04349253]]


dL/dPred [0.08088688]
dPred/dH [[-0.12488763  0.1282223 ]]
dH/dW [[3.71924713e-13 8.92619312e-14 0.00000000e+00 0.00000000e+00
  1.48769885e-14 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 1.22135668e-10 2.93125603e-11
  0.00000000e+00 4.88542671e-12]]
dPred/dW [[3.88077679e-02 1.89592506e-13 3.88077679e-02]]


dL/dPred [0.08083817]
dPred/dH [[-0.12482778  0.12814834]]
dH/dW [[7.01195324e-09 1.64987135e-09 0.00000000e+00 0.00000000e+00
  4.12467837e-10 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 4.49539571e-07 1.05774017e-07
  0.00000000e+00 2.64435042e-08]]
dPred/dW [[3.87853838e-02 1.02562149e-09 3.87853838e-02]]


dL/dPred [-0.06746677]
dPred/dH [[-0.10491615  0.10769655]]
dH/dW [[-7.56532978e-09 -3.02613191e-09  0.

dPred/dH [[-0.10406325  0.10680259]]
dH/dW [[-7.18979039e-09 -2.87591615e-09  0.00000000e+00  0.00000000e+00
   4.79319359e-10  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00 -3.68069148e-07 -1.47227659e-07
   0.00000000e+00  2.45379432e-08]]
dPred/dW [[1.54305368e-11 3.21926001e-02 3.21926009e-02]]


Epoch 1000 loss: 0.006
dL/dPred [-0.08983949]
dPred/dH [[-0.13868147  0.14234129]]
dH/dW [[-0.09455081 -0.0472754   0.          0.          0.0472754   0.        ]
 [ 0.          0.         -0.08686575 -0.04343288  0.          0.04343288]]
dPred/dW [[0.00213439 0.04094977 0.04290196]]


dL/dPred [0.07975617]
dPred/dH [[-0.12376545  0.12704647]]
dH/dW [[3.44169138e-13 8.26005930e-14 0.00000000e+00 0.00000000e+00
  1.37667655e-14 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 1.13376907e-10 2.72104577e-11
  0.00000000e+00 4.53507628e-12]]
dPred/dW [[3.82878243e-02 1.73638204e-13 3.82878243e-02]]


dL/dPred [0.07970942]
dPred/dH [[-0.1237076  0.1269751]]
dH/dW [[6.64937638e-09 1.56455915

dL/dPred [0.07875907]
dPred/dH [[-0.12276796  0.12600178]]
dH/dW [[3.21964677e-13 7.72715225e-14 0.00000000e+00 0.00000000e+00
  1.28785871e-14 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 1.06110827e-10 2.54665986e-11
  0.00000000e+00 4.24443309e-12]]
dPred/dW [[3.78287868e-02 1.60561754e-13 3.78287868e-02]]


dL/dPred [0.07871401]
dPred/dH [[-0.12271186  0.12593264]]
dH/dW [[6.34251496e-09 1.49235646e-09 0.00000000e+00 0.00000000e+00
  3.73089115e-10 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 4.08955312e-07 9.62247793e-08
  0.00000000e+00 2.40561948e-08]]
dPred/dW [[3.78080290e-02 9.09517333e-10 3.78080290e-02]]


dL/dPred [-0.06577176]
dPred/dH [[-0.10323561  0.10593549]]
dH/dW [[-6.84306007e-09 -2.73722403e-09  0.00000000e+00  0.00000000e+00
   4.56204005e-10  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00 -3.51306344e-07 -1.40522538e-07
   0.00000000e+00  2.34204229e-08]]
dPred/dW [[1.45092950e-11 3.18044000e-02 3.18044008e-02]]


Epoch 1000 loss: 0.006
dL/dPred [-0.088

In [1295]:
emily = np.array([-7, -3]) # 128 pounds, 63 inches
frank = np.array([20, 2])  # 155 pounds, 68 inches
p1,h1 = network.feedforward(emily)
p2,h2 = network.feedforward(frank)
print("Emily: %.3f" % p1[0]) # 0.951 - F
print("Frank: %.3f" % p2[0]) # 0.039

Emily: 0.736
Frank: 0.881
